# Initialize

In [1]:
import os
init=False

# Imports

In [2]:
if not init:
    os.chdir('..')
    init=True
from pythonfigures.datapartition import DataPartitioner
from pythonfigures.neuraldatabase import Query
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from scipy.linalg import subspace_angles, null_space, orth

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import colors

from typing import Optional

# Get Data

In [3]:
dp = DataPartitioner(session='Moe46',
                    areas=['AIP'],
                    aligns=['fixation','cue onset','go cue onset','movement onset','hold onset'],
                    contexts=['active','passive','control'],
                    groupings=['context','alignment','grip','object','turntable','time'])

# TODO: [x] try go cue onset as the alignment for the "visual" space (i.e., include memory signals, *really* make sure you're isolating those transients!) ([x] also include context & turntable as separable factors in the space you try to remove, and use a 95% variance criterion like the one used in the paper instead of these weak classification criteria)
# uh oh! using go cue onset actually preferentially affects VGG performance, with little effect on Obs! All while doubling the number of axes we remove!
# moreover, these measures either have a UNIFORM or cross-context effect or no effect at all (regardless, still no preferential reduction in passive classification accuracy) when considering just the mixed turntable
# and we get better results for the special turntable control, too (although just barely... it still doesn't jibe with intuition in that there's still substantial classification accuracy even after removing the big fat premovement space)
# in any case the cross-decoding controls are unaffected regardless
# THEREFORE: just use the visual subspace control
# (note: all the above is based on AIP of Moe46, Moe50, Zara70, F5 of Zara70)
# (it's also true for the main result of F5 in Moe46, Moe 50, but with selective, albeit *very* slight, reduction in passive performance on the mixed turntable)
# (note: these measures actually have the desired effect in AIP, F5 of Zara64)
# (Zara68 doesn't have enough neurons...)

# TODO: (but: we have one last trick up our sleeve. remove the space that interferes the least with the target decoding space!)
# except wait... this ain't gonna work... unless we bake in the assumption that all we care about is the active context, which feels pretty funny
# plus we need to either develop a new method or set some kind of arbitrary threshold on the eigenvalues of the difference-of-covariances problem... yeah okay screw it)

# TODO: okay the REAL last stand: preferentially preserve time-varying over static components
# use the pandas "transform" method!
# nope! this also doesn't appreciably change the results from the visual case! It preserves more movement-period information but also doesn't detract from encoding during the passive context...
# stop getting fancy. just use the visual subspace trick, show you still get decent accuracy, but ***report on the subspace alignment*** and ***subsampling results*** afterward!

# (not now) TODO: visualize the time-varying nature of the signals before & after removal... *somehow* (save this for after NCM)

# TODO: [x] active vs. passive cross-classification of OBJECT, before & after subspace removal (at least this tracks)

In [4]:
# query the whole damn thing
df = dp.readQuery(0)

# convert from turntable x object ID vs. just the turntable ID
df['turntable'] = df['turntable'] // 10

# Here be controls:
1. Vision-Grasping cross-decoding is abolished
2. Special turntable stops being decoded during movement
3. grasping box keeps being decoded during movement
4. "control" task performance during movement is less affected than "active" task

## Vision-Grasping cross-decoding

In [20]:
np.eye(2)

array([[1., 0.],
       [0., 1.]])

In [5]:
# vision-grasping cross-decoding
df_vision_trials   = df[(df['alignment']=='go cue onset') & (df['time']<0)].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
df_movement_trials = df[(df['alignment']=='hold onset') & (df['time']<0)].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
df_movement        = df[(df['alignment']=='hold onset') & (df['time']<0)].groupby(['object','trial','time'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
labels             = df_movement_trials.index.get_level_values(0)

assert np.all( df_vision_trials.index.get_level_values(-1).to_numpy() == df_movement_trials.index.get_level_values(-1).to_numpy() ), 'trials not lined up between vision & movement!'

flag          = False
neurcount     = len(dp.get('neuronColumnNames'))
pccount       = 30
nsplits       = 5

skf     = StratifiedKFold(n_splits=nsplits,shuffle=False)
splits  = skf.split(np.zeros(len(labels)),labels)
splits  = list(splits) # store as list to allow it to be used many of time
spaces2null = []
deltamu     = []

# step 1: find the set of axes for each split
for train_,_ in splits:
    timevary_ = df_movement.groupby(level=[0,2]).aggregate('mean')
    static_   = timevary_.groupby(level=[0]).transform('mean')
    
    delta_    = timevary_.to_numpy() - static_.to_numpy()
    
    PCmdl = PCA()
    PCmdl.fit(delta_)
    
    cvals = np.cumsum(PCmdl.explained_variance_ratio_)
    idx=0
    while cvals[idx]<0.95:
        idx+=1
    
    spaces2null+=[PCmdl.components_[:idx,:]]
    
    # also get delta-mus
    deltamu += [df_vision_trials.iloc[train_].mean() - df_movement_trials.iloc[train_].mean()]

# step 2: model fitting
for removeAxes in [False,True]:
    correct_count = 0
    chance_count  = 0
    total_count   = len(labels)
    
    for fold,(train_,test_) in enumerate(splits):
        trainX = df_vision_trials.iloc[train_]
        trainy = labels[train_].to_numpy()
        
        testX  = df_movement_trials.iloc[test_]
        testy  = labels[test_].to_numpy()
        
        if removeAxes:
            nullSpace = null_space( np.matrix(spaces2null[fold]) )
        else:
            nullSpace = np.eye(neurcount)
            
        # projections
        trainX = trainX.to_numpy() @ nullSpace
        testX  = testX.to_numpy() @ nullSpace
        
        # now get rid of deltamu, too (this improves model generalization across epochs)
        deltaNull = null_space(np.matrix(deltamu[fold].to_numpy() @ nullSpace))
        
        # now fit a PC model in the remaining space to make sure LDA input is well-conditioned (this improves cross-validated performance)
        PCmdl = PCA(n_components=pccount)
        trainX = PCmdl.fit_transform(trainX @ deltaNull)
        testX  = PCmdl.transform(testX @ deltaNull)
        
        # now we can finally start classifying!
        LDmdl = LDA()
        LDmdl.fit(trainX,trainy)
        yhat = LDmdl.predict(testX)
        
        correct_count += np.sum(yhat==testy)
        chance_count  += len(testy) * max(LDmdl.priors_)
        
    correct_rate   = correct_count / total_count
    chance_rate    = chance_count / total_count
    threshold_rate = chance_rate + np.sqrt( chance_rate*(1-chance_rate)/total_count ) # chance + 1 SE
    
    if removeAxes:
        print(f"Removed axes = {np.mean( [x.shape[0] for x in spaces2null] )}")
    else:
        print(f"Removed axes = { 0 }")
    print(f"Accuracy = {correct_rate}")
    print(f"Chance = {chance_rate}")
    print(f"Threshold = {threshold_rate}")
    print()
    
# raw cross-accuracy: roughly 40% for active, 25% for passive (for one of Moe's sessions in AIP)

Removed axes = 0
Accuracy = 0.37037037037037035
Chance = 0.09413473478984076
Threshold = 0.10560620950962894

Removed axes = 40.0
Accuracy = 0.3410493827160494
Chance = 0.09413473478984076
Threshold = 0.10560620950962894



## Special turntable

In [6]:
if dp.get('session')!='Zara70':
    # special turntable control
    df_vision_included = df[(df['alignment']=='go cue onset') & (df['time']<0) & (df['turntable']==9) & (df['context']=='active')].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_vision_excluded = df[(df['alignment']=='go cue onset') & (df['time']<0) & ((df['turntable']!=9) | (df['context']!='active')) ].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_movement_trials = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']==9) & (df['context']=='active')].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    labels             = df_movement_trials.index.get_level_values(0)

    assert np.all( df_vision_included.index.get_level_values(-1).to_numpy() == df_movement_trials.index.get_level_values(-1).to_numpy() ), 'trials not lined up between vision & movement!'

    flag          = False
    neurcount     = len(dp.get('neuronColumnNames'))
    pccount       = 30
    nsplits       = 5

    skf     = StratifiedKFold(n_splits=nsplits,shuffle=False)
    splits  = skf.split(np.zeros(len(labels)),labels)
    splits  = list(splits) # store as list to allow it to be used many of time
    spaces2null = []

    # step 1: find the set of axes for each split
    for train_,_ in splits:
        df_vision_cat   = pd.concat( (df_vision_excluded,df_vision_included.iloc[train_]) )
        df_vision_agg   = df_vision_cat.groupby(level=[0,1,2]).aggregate('mean')
        df_movement_agg = df_movement_trials.iloc[train_].groupby(level=[0]).aggregate('mean')
        
        viscov = np.cov(df_vision_agg.to_numpy(),rowvar=False)
        movcov = np.cov(df_movement_agg.to_numpy(),rowvar=False)

        vals,vecs = np.linalg.eigh(movcov-viscov) # sorts ascending, and we want to preferentially list the vision-preferring dimensions first

        idx=0
        while vals[idx]<-1e-6:
            idx+=1

        spaces2null+=[vecs[:,:idx].T]

        # no need for deltamu, as we're not doing cross-decoding!

    # step 2: model fitting
    for removeAxes in [False,True]:
        correct_count = 0
        chance_count  = 0
        total_count   = len(labels)

        for fold,(train_,test_) in enumerate(splits):
            trainX = df_movement_trials.iloc[train_]
            trainy = labels[train_].to_numpy()

            testX  = df_movement_trials.iloc[test_]
            testy  = labels[test_].to_numpy()

            if removeAxes:
                nullSpace = null_space( np.matrix(spaces2null[fold]) )
            else:
                nullSpace = np.eye(neurcount)

            # projections
            trainX = trainX.to_numpy() @ nullSpace
            testX  = testX.to_numpy() @ nullSpace

            # no deltamu to get rid of!

            # now fit a PC model in the remaining space to make sure LDA input is well-conditioned (this improves cross-validated performance)
            PCmdl = PCA(n_components=pccount)
            trainX = PCmdl.fit_transform(trainX)
            testX  = PCmdl.transform(testX)

            # now we can finally start classifying!
            LDmdl = LDA()
            LDmdl.fit(trainX,trainy)
            yhat = LDmdl.predict(testX)

            correct_count += np.sum(yhat==testy)
            chance_count  += len(testy) * max(LDmdl.priors_)

        correct_rate   = correct_count / total_count
        chance_rate    = chance_count / total_count
        threshold_rate = chance_rate + np.sqrt( chance_rate*(1-chance_rate)/total_count ) # chance + 1 SE

        if removeAxes:
            print(f"Removed axes = {np.mean( [x.shape[0] for x in spaces2null] )}")
        else:
            print(f"Removed axes = { 0 }")    
        print(f"Accuracy = {correct_rate}")
        print(f"Chance = {chance_rate}")
        print(f"Threshold = {threshold_rate}")
        print()

Removed axes = 0
Accuracy = 0.8169014084507042
Chance = 0.17610046242366476
Threshold = 0.221305642476746

Removed axes = 53.0
Accuracy = 0.7746478873239436
Chance = 0.17610046242366476
Threshold = 0.221305642476746



## Control task

In [7]:
# control task
# hmmmm it doesn't seem like VGG is really all that adversely affected by removal of the visual space
# or Obs for that matter...
# note: object & grip decoding should be the same for the "mixed" turntable (they are for Moe50 in any case)

flag          = False
neurcount     = len(dp.get('neuronColumnNames'))
pccount       = 30
nsplits       = 5

for context_ in ['active','control','passive']:
    print(context_.upper())
    df_vision_included = df[(df['alignment']=='go cue onset') & (df['time']<0) & ((df['turntable']==1) & (df['context']==context_))].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_vision_excluded = df[(df['alignment']=='go cue onset') & (df['time']<0) & ((df['turntable']!=1) | (df['context']!=context_)) ].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    
    df_classify = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']==1) & (df['context']==context_)].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_cross    = df[(df['alignment']=='hold onset') & (df['time']<0) & ((df['turntable']!=1) | (df['context']!=context_))].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    labels      = df_classify.index.get_level_values(0)
    
    skf     = StratifiedKFold(n_splits=nsplits,shuffle=False)
    splits  = skf.split(np.zeros(len(labels)),labels)
    splits  = list(splits) # store as list to allow it to be used many of time
    spaces2null = []

    # step 1: find the set of axes for each split
    for train_,_ in splits:
        df_vision_cat   = pd.concat( (df_vision_excluded,df_vision_included.iloc[train_]) )
        df_vision_agg   = df_vision_cat.groupby(level=[0,1,2]).aggregate('mean')
        df_movement_cat = pd.concat( (df_cross,df_classify.iloc[train_]) )
        df_movement_agg = df_movement_cat.groupby(level=[0]).aggregate('mean')
        
        viscov = np.cov(df_vision_agg.to_numpy(),rowvar=False)
        movcov = np.cov(df_movement_agg.to_numpy(),rowvar=False)

        vals,vecs = np.linalg.eigh(movcov-viscov) # sorts ascending, and we want to preferentially list the vision-preferring dimensions first

        idx=0
        while vals[idx]<-1e-6:
            idx+=1

        spaces2null+=[vecs[:,:idx].T]

        # no need for deltamu, as we're not doing cross-decoding!

    # step 2: model fitting
    for removeAxes in [False,True]:
        correct_count = 0
        chance_count  = 0
        total_count   = len(labels)

        for fold,(train_,test_) in enumerate(splits):
            trainX = df_classify.iloc[train_]
            trainy = labels[train_].to_numpy()

            testX  = df_classify.iloc[test_]
            testy  = labels[test_].to_numpy()

            if removeAxes:
                nullSpace = null_space( np.matrix(spaces2null[fold]) )
            else:
                nullSpace = np.eye(neurcount)

            # projections
            trainX = trainX.to_numpy() @ nullSpace
            testX  = testX.to_numpy() @ nullSpace

            # no deltamu to get rid of!

            # now fit a PC model in the remaining space to make sure LDA input is well-conditioned (this improves cross-validated performance)
            PCmdl = PCA(n_components=pccount)
            trainX = PCmdl.fit_transform(trainX)
            testX  = PCmdl.transform(testX)

            # now we can finally start classifying!
            LDmdl = LDA()
            LDmdl.fit(trainX,trainy)
            yhat = LDmdl.predict(testX)

            correct_count += np.sum(yhat==testy)
            chance_count  += len(testy) * max(LDmdl.priors_)

        correct_rate   = correct_count / total_count
        chance_rate    = chance_count / total_count
        threshold_rate = chance_rate + np.sqrt( chance_rate*(1-chance_rate)/total_count ) # chance + 1 SE

        if removeAxes:
            print(f"    Removed axes = {np.mean( [x.shape[0] for x in spaces2null] )}")
        else:
            print(f"    Removed axes = { 0 }")    
        print(f"    Accuracy = {correct_rate}")
        print(f"    Chance = {chance_rate}")
        print(f"    Threshold = {threshold_rate}")
        print()

ACTIVE
    Removed axes = 0
    Accuracy = 0.8831168831168831
    Chance = 0.1786280879353328
    Threshold = 0.22227961446074423

    Removed axes = 53.0
    Accuracy = 0.8831168831168831
    Chance = 0.1786280879353328
    Threshold = 0.22227961446074423

CONTROL
    Removed axes = 0
    Accuracy = 0.8285714285714286
    Chance = 0.17857142857142858
    Threshold = 0.22434785317583256

    Removed axes = 53.0
    Accuracy = 0.9142857142857143
    Chance = 0.17857142857142858
    Threshold = 0.22434785317583256

PASSIVE
    Removed axes = 0
    Accuracy = 0.7638888888888888
    Chance = 0.17367412784835654
    Threshold = 0.2183195558157913

    Removed axes = 53.0
    Accuracy = 0.6805555555555556
    Chance = 0.17367412784835654
    Threshold = 0.2183195558157913



# full result (excluding the special turntable due to the lack of grip variety)

In [8]:
df_active_trials   = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']!=9) & (df['context']=='active')].groupby(['grip','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
df_passive_trials  = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']!=9) & (df['context']=='passive')].groupby(['grip','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
active_labels      = df_active_trials.index.get_level_values(0)
passive_labels     = df_passive_trials.index.get_level_values(0)

flag          = False
# axesToRemove  = 0 # don't undo your hard work!
neurcount     = len(dp.get('neuronColumnNames'))
pccount       = 30
nsplits       = 5

for context_ in ['active','passive']:
    print(context_.upper())
    df_vision_included = df[(df['alignment']=='go cue onset') & (df['time']<0) & ((df['turntable']!=9) & (df['context']==context_) & (df['grip'].notna()))].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_vision_excluded = df[(df['alignment']=='go cue onset') & (df['time']<0) & ((df['turntable']==9) | (df['context']!=context_) | (df['grip'].isna())) ].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    
    if context_=='active':
        labels = active_labels
        df_classify = df_active_trials
        df_movement = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']!=9) & (df['context']=='active')].groupby(['grip','trial','time'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    elif context_=='passive':
        labels = passive_labels
        df_classify = df_passive_trials  
        df_movement = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['turntable']!=9) & (df['context']=='passive')].groupby(['grip','trial','time'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    
    skf     = StratifiedKFold(n_splits=nsplits,shuffle=False)
    splits  = skf.split(np.zeros(len(labels)),labels)
    splits  = list(splits) # store as list to allow it to be used many of time
    spaces2null = []

    # step 1: find the set of axes for each split
    for train_,_ in splits:
        timevary_ = df_movement.groupby(level=[0,2]).aggregate('mean')
        static_   = timevary_.groupby(level=[0]).transform('mean')

        delta_    = timevary_.to_numpy() - static_.to_numpy()

        PCmdl = PCA()
        PCmdl.fit(delta_)

        cvals = np.cumsum(PCmdl.explained_variance_ratio_)
        idx=0
        while cvals[idx]<0.95:
            idx+=1

        spaces2null+=[PCmdl.components_[:idx,:]]

        # no need for deltamu, as we're not doing cross-decoding!

    # step 2: model fitting
    for removeAxes in [False,True]:
        correct_count = 0
        chance_count  = 0
        total_count   = len(labels)

        for fold,(train_,test_) in enumerate(splits):
            trainX = df_classify.iloc[train_]
            trainy = labels[train_].to_numpy()

            testX  = df_classify.iloc[test_]
            testy  = labels[test_].to_numpy()

            if removeAxes:
                nullSpace = null_space( np.matrix(spaces2null[fold]) )
            else:
                nullSpace = np.eye(neurcount)

            # projections
            trainX = trainX.to_numpy() @ nullSpace
            testX  = testX.to_numpy() @ nullSpace

            # no deltamu to get rid of!

            # now fit a PC model in the remaining space to make sure LDA input is well-conditioned (this improves cross-validated performance)
            PCmdl = PCA(n_components=pccount)
            trainX = PCmdl.fit_transform(trainX)
            testX  = PCmdl.transform(testX)

            # now we can finally start classifying!
            LDmdl = LDA()
            LDmdl.fit(trainX,trainy)
            yhat = LDmdl.predict(testX)

            correct_count += np.sum(yhat==testy)
            chance_count  += len(testy) * max(LDmdl.priors_)

        correct_rate   = correct_count / total_count
        chance_rate    = chance_count / total_count
        threshold_rate = chance_rate + np.sqrt( chance_rate*(1-chance_rate)/total_count ) # chance + 1 SE

        if removeAxes:
            print(f"    Removed axes = {np.mean( [x.shape[0] for x in spaces2null] )}")
        else:
            print(f"    Removed axes = { 0 }")    
        print(f"    Accuracy = {correct_rate}")
        print(f"    Chance = {chance_rate}")
        print(f"    Threshold = {threshold_rate}")
        print()

ACTIVE
    Removed axes = 0
    Accuracy = 0.75
    Chance = 0.22272727272727272
    Threshold = 0.25077915928761463

    Removed axes = 22.0
    Accuracy = 0.740909090909091
    Chance = 0.22272727272727272
    Threshold = 0.25077915928761463

PASSIVE
    Removed axes = 0
    Accuracy = 0.5576923076923077
    Chance = 0.23079404466501238
    Threshold = 0.2645283148467347

    Removed axes = 25.0
    Accuracy = 0.44871794871794873
    Chance = 0.23079404466501238
    Threshold = 0.2645283148467347



# try object cross-classification across contexts, at least prove that *this* is entirely a visual effect

In [9]:
df_active_trials   = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['context']=='active')].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
df_passive_trials  = df[(df['alignment']=='hold onset') & (df['time']<0) & (df['context']=='passive')].groupby(['object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
active_labels      = df_active_trials.index.get_level_values(0)
passive_labels     = df_passive_trials.index.get_level_values(0)

for context_ in ['active','passive']:
    print(f"Test context = {context_.upper()}")
    df_vision_included = df[(df['alignment']=='go cue onset') & (df['time']<0) & (df['context']==context_)].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    df_vision_excluded = df[(df['alignment']=='go cue onset') & (df['time']<0) & (df['context']!=context_)].groupby(['context','turntable','object','trial'],sort=False)[dp.get('neuronColumnNames')].aggregate('mean')
    
    if context_=='active':
        labels = active_labels
        df_classify = df_active_trials
        df_cross    = df_passive_trials
        crosslabels = passive_labels
    elif context_=='passive':
        labels = passive_labels
        df_classify = df_passive_trials
        df_cross    = df_active_trials
        crosslabels = active_labels
    
    skf     = StratifiedKFold(n_splits=nsplits,shuffle=False)
    splits  = skf.split(np.zeros(len(labels)),labels)
    splits  = list(splits) # store as list to allow it to be used many of time
    spaces2null = []
    deltamu     = []

    # step 1: find the set of axes for each split
    for train_,_ in splits:
        df_vision_cat = pd.concat( (df_vision_excluded,df_vision_included.iloc[train_]) )
        df_vision_agg = df_vision_cat.groupby(level=[0,1,2]).aggregate('mean')
        PCmdl = PCA()
        PCmdl.fit(df_vision_agg.to_numpy())

        idx=0
        vexp=PCmdl.explained_variance_ratio_[0]
        while vexp<0.95:
            idx+=1
            vexp+=PCmdl.explained_variance_ratio_[idx]

        spaces2null+=[PCmdl.components_[:idx,:]]

        # no need for deltamu, as we're not doing cross-decoding!
        deltamu += [df_classify.iloc[train_].mean() - df_cross.mean()]

    # step 2: model fitting
    for removeAxes in [False,True]:
        correct_count = 0
        chance_count  = 0
        total_count   = len(labels)

        for fold,(train_,test_) in enumerate(splits):
            trainX = df_cross
            trainy = crosslabels.to_numpy()

            testX  = df_classify.iloc[test_]
            testy  = labels[test_].to_numpy()

            if removeAxes:
                nullSpace = null_space( np.matrix(spaces2null[fold]) )
            else:
                nullSpace = np.eye(neurcount)

            # projections
            trainX = trainX.to_numpy() @ nullSpace
            testX  = testX.to_numpy() @ nullSpace

            # no deltamu to get rid of!

            # now fit a PC model in the remaining space to make sure LDA input is well-conditioned (this improves cross-validated performance)
            PCmdl = PCA(n_components=pccount)
            trainX = PCmdl.fit_transform(trainX)
            testX  = PCmdl.transform(testX)

            # now we can finally start classifying!
            LDmdl = LDA()
            LDmdl.fit(trainX,trainy)
            yhat = LDmdl.predict(testX)

            correct_count += np.sum(yhat==testy)
            chance_count  += len(testy) * max(LDmdl.priors_)

        correct_rate   = correct_count / total_count
        chance_rate    = chance_count / total_count
        threshold_rate = chance_rate + np.sqrt( chance_rate*(1-chance_rate)/total_count ) # chance + 1 SE

        if removeAxes:
            print(f"    Removed axes = {np.mean( [x.shape[0] for x in spaces2null] )}")
        else:
            print(f"    Removed axes = { 0 }")    
        print(f"    Accuracy = {correct_rate}")
        print(f"    Chance = {chance_rate}")
        print(f"    Threshold = {threshold_rate}")
        print()

Test context = ACTIVE
    Removed axes = 0
    Accuracy = 0.3402061855670103
    Chance = 0.08362369337979093
    Threshold = 0.09985132363803655

    Removed axes = 26.4
    Accuracy = 0.07216494845360824
    Chance = 0.08362369337979093
    Threshold = 0.09985132363803655

Test context = PASSIVE
    Removed axes = 0
    Accuracy = 0.18466898954703834
    Chance = 0.0859106529209622
    Threshold = 0.10245222894643412

    Removed axes = 26.8
    Accuracy = 0.06620209059233449
    Chance = 0.0859106529209622
    Threshold = 0.10245222894643412

